Grupo 10: Juan Diego Perez, Diana Bayona, Jorge Rodriguez, Santiago Gutiérrez

El propósito de este proyecto es poder poner en práctica los conocimientos sobre técnicas de preprocesamiento, modelos predictivos de NLP, y la disponibilización de modelos.

## Datos para la predicción de género en películas

En este proyecto se usará un conjunto de datos de géneros de películas. Cada observación contiene el título de una película, su año de lanzamiento, la sinopsis o plot de la película (resumen de la trama) y los géneros a los que pertenece (una película puede pertenercer a más de un género). Por ejemplo:
- Título: 'How to Be a Serial Killer'
- Plot: 'A serial killer decides to teach the secrets of his satisfying career to a video store clerk.'
- Generos: 'Comedy', 'Crime', 'Horror'

La idea es que usen estos datos para predecir la probabilidad de que una película pertenezca, dada la sinopsis, a cada uno de los géneros. Agradecemos al profesor Fabio González, Ph.D. y a su alumno John Arevalo por proporcionar este conjunto de datos. Ver https://arxiv.org/abs/1702.01992

## Librerias

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score

from werkzeug.utils import cached_property
import seaborn as sns
import joblib
from flask import Flask
#from flask_restplus import Api, Resource, fields
from flask_restx import Api, Resource, fields

import os
os.chdir('..')

## Cargue de Información

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


## 1. Preprocesamiento
Los datos de entrenamiento se dividen en datos de entrenamiento y validación. Si decidieron preprocesar los datos (estandarizar, normalizar, imputar valores, etc), estos son correctamente preprocesados al ajustar sobre los datos de entrenamiento (.fit_transform()) y al transformar los datos del set de validación (.transform()).

### 1.1 Vectorización del Campo Texto

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Santiago/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Definición de variables predictoras (X)
vect = TfidfVectorizer(stop_words=english_stop_words, lowercase=True, ngram_range=(1, 6),max_features=5000)
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 5000)

In [8]:
# Definición de variable de interés (y)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

Para la parte de preprocesamiento utilizamos la función **TfidVectorizer** ya que que es mucho más efectiva para capturar la información relevante en el texto, reducir el ruido y mejorar la calidad de las características, haciendo que las palabras realmente importantes se destaquen. **TfidVectorizer** pondera la importancia relativa de los términos en función de su frecuencia en el documento. Ademas, contrario a la función **CountVectorizer**, la función TfidVectorizer tiende a mejorar el rendimiento de los modelos de ML en tareas de clasificación, ya que como hemos señalado proporciona una representación más informativa y diferenciada de los textos. Al final, esta función convierte el texto en una matriz de números para que los modelos las pueden entender y utilizar.

Dentro de la función **TfidVectorizer** podemos cambiar o seleccionar algunos parámetros que nos permiten hacer un mejor preprocesamiento. En este ejercicio, eliminamos las palabras comunes que no aportan significado al texto, conocidas como 'stop_words', se eliminan palabras como: "the", "of", "are", "and". Convertimos el texto a minúsculas y le pedimos que realizara un análisis por n-gramas, analizando en este caso secuencias de 1 a 6 palabras consecutivas. Por último, limitamos el número de palabras únicas a las 5000 más comunes. 
    
En resumen, los datos textuales de la variable plot, que contienen una breve sinopsis de cada una de las películas y que son el principal insumo para ayudarnos a predecir la probabilidad de que una pelicula pertenezca a uno o varios géneros específicos, quedan preparados para que los modelos que utilizaremos en el siguiente paso puedan procesarlos. Este preprocesamiento nos proporciona una matriz numérica de 7895 filas y 5000 columnas.

## 1.2 División Conjunto de Datos

In [9]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

De la bases de datos se separa la variable a predecir de sus predictoras. Luego los datos se dividen en dos muestras, una de entrenamiento y otra de validación. La muestra de entrenamiento contiene el 67% de la información y la de validación el 33% restante. Dividir los datos de esta manera nos permite ver que tambien se desempeña el modelo con información por fuera de la que utiliza para poder entrenarse.

## 2. Calibración del Modelo 
Se calibran los parámetros que se consideren pertinentes del modelo de clasificación seleccionado. Se justifica el método seleccionado de calibración. Se analizan los valores calibrados de cada parámetro y se explica cómo afectan el modelo.

In [13]:
# Definir los modelos y sus parámetros
models = [
    {
        'name': 'Naive Bayes',
        'estimator': OneVsRestClassifier(MultinomialNB()),
        'param_grid': {
            'clf__estimator__alpha': [0.01, 0.1, 0.15, 0.20, 0.25, 0.5, 0.75, 1.0]
        }
    },
    {
        'name': 'SVM',
        'estimator': OneVsRestClassifier(SVC(probability=True)),
        'param_grid': {
            'clf__estimator__C': [0.1, 1],
            'clf__estimator__kernel': ['rbf', 'sigmoid'],
            'clf__estimator__gamma' : ['scale', 'auto']
        }
    },
    {
        'name': 'Random Forest',
        'estimator': OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, random_state=42)),
        'param_grid': {
            'clf__estimator__n_estimators': [200, 300],
            'clf__estimator__min_samples_split': [5, 10],
            'clf__estimator__min_samples_leaf': [2, 4],
            'clf__estimator__bootstrap': [True, False]
        }
    },
    {
        'name': 'XGBoost',
        'estimator':OneVsRestClassifier(XGBClassifier(objective='binary:logistic', eval_metric='auc', use_label_encoder=False, random_state=42)),
        'param_grid': {
            'clf__estimator__n_estimators': [200, 300],
            'clf__estimator__learning_rate': [0.01, 0.05, 0.1, 0.5],
            'clf__estimator__gamma': [0, 0.5, 1],
            'clf__estimator__colsample_bytree': [0.5, 1.0],
        }
    }
]

In [14]:
def evaluate_model(X_train, y_train_genres, X_test, y_test_genres, models):
    results = []
    
    for model in models:
        print(f"Evaluando el modelo: {model['name']}")
        
        pipeline = Pipeline([('clf', model['estimator'])])
        roc_auc_scorer = make_scorer(roc_auc_score, average='macro', multi_class='ovr', needs_proba=True)# Crear el scorer para roc_auc
        grid_search = GridSearchCV(estimator=pipeline, param_grid=model['param_grid'], 
                                   scoring=roc_auc_scorer, cv=3, verbose=2,n_jobs=-1) # Configurar y ejecutar GridSearchCV
        
        grid_search.fit(X_train, y_train_genres) # Entrenar el modelo
        best_model = grid_search.best_estimator_ # Obtener el mejor modelo
        y_pred_proba = best_model.predict_proba(X_test) # Predecir probabilidades en el conjunto de prueba
        roc_auc = roc_auc_score(y_test_genres, y_pred_proba, average='macro') # Evaluar el modelo en el conjunto de prueba usando roc_auc_score
        
        # Guardar los resultados
        results.append({
            'model': model['name'],
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'roc_auc': roc_auc
        })
        
        # Mostrar los mejores parámetros y la puntuación
        print(f"Mejores parámetros para {model['name']}: {grid_search.best_params_}")
        print(f"Mejor puntuación de validación cruzada para {model['name']}: {grid_search.best_score_}")
        print(f"ROC AUC Score en el conjunto de prueba para {model['name']}: {roc_auc}\n")
        
    return results

In [ ]:
# Evaluar todos los modelos
results = evaluate_model(X_train, y_train_genres, X_test, y_test_genres, models)

# Mostrar los resultados
for result in results:
    print(f"Modelo: {result['model']}")
    print(f"Mejores parámetros: {result['best_params']}")
    print(f"Mejor puntuación de validación cruzada: {result['best_score']}")
    print(f"ROC AUC Score en el conjunto de prueba: {result['roc_auc']}\n")

Modelo: Naive Bayes
Mejores parámetros: {'clf__estimator__alpha': 0.1}
Mejor puntuación de validación cruzada: 0.8503826776957646
ROC AUC Score en el conjunto de prueba: 0.8657962945897676

Modelo: SVM
Mejores parámetros: {'clf__estimator__C': 0.1, 'clf__estimator__gamma': 'scale', 'clf__estimator__kernel': 'rbf'}
Mejor puntuación de validación cruzada: 0.8391160456109027
ROC AUC Score en el conjunto de prueba: 0.8624744738664503

Modelo: Random Forest
Mejores parámetros: {'clf__estimator__bootstrap': True, 'clf__estimator__min_samples_leaf': 4, 'clf__estimator__min_samples_split': 10, 'clf__estimator__n_estimators': 300}
Mejor puntuación de validación cruzada: 0.8278230550291058
ROC AUC Score en el conjunto de prueba: 0.8330338942648683

Modelo: XGBoost
Mejores parámetros: {'clf__estimator__colsample_bytree': 0.5, 'clf__estimator__gamma': 0.5, 'clf__estimator__learning_rate': 0.05, 'clf__estimator__n_estimators': 300}
Mejor puntuación de validación cruzada: 0.8158000755448915
ROC AUC Score en el conjunto de prueba: 0.8288969614295976

El problema consiste en encontrar la probabilidad de que una película pertenezca a uno o varios géneros. Entonces, consiste en un problema de clasificación multietiqueta, donde una pelicula puede pertenecer a múltiples generos. Entre los modelos que ayudan a resolver este tipo de problemas y que son utilizados en este ejercicio se encuentran el Clasificador Naive de Bayes, las Máquinas de Soporte Vectorial (SVM), Random Forest y XGBoost. Para cada modelo, creamos un conjunto de diferentes hiperparámetros y los calibramos utilizando la función GridSearchCV de scikit-learn. Esta función busca la mejor combinación de hiperparámetros mediante una búsqueda exhaustiva, optimizando una función objetivo, en este caso, buscando el mejor AUC (Área Bajo la Curva). Utilizamos la validación cruzada para evaluar el rendimiento de cada combinación de hiperparámetros y asegurar la robustez de los resultados.

Se comparan los resultados de todos los modelos y el resultado final es que el modelo con mejor desempeño, de acuerda a la metrica AUC, es el **Clasificador Naive de Bayes**. Los hiperparametros del modelo que se calibraron son: **alpha.** 

En un modelo de clasificación Naive Bayes, alpha es un parámetro de suavizado. En este caso, el suavizado se utiliza para manejar el problema de las probabilidades cero dentro del modelo. Esto ocurre cuando una palabra en el conjunto de datos de prueba no aparece en el conjunto de datos de entrenamiento, lo que resultaría en una probabilidad cero y podría generar sesgos al modelo. Entonces, alpha agrega un valor constante a todas las frecuencias de palabras para asegurar que ninguna probabilidad sea cero.

Un valor de alpha igual a 0.1 es relativamente pequeño, lo que significa que se está aplicando un suavizado ligero. Esto ayuda a corregir las probabilidades sin alterar significativamente las frecuencias originales de las palabras. Por lo tanto, este valor implica que el modelo ha logrado encontrar un equilibrio entre adaptarse a los datos de entrenamiento sin ser demasiado vulnerable a las nuevas palabras en los datos de prueba.

## 3. Entrenamiento del Modelo
Se entrena el modelo de clasificación escogido con los datos del set de entrenamiento preprocesados y los parámetros óptimos. Se presenta el desempeño del modelo en los datos de validación con al menos una métrica de desempeño. Se justifica la selección del modelo correctamente.

In [20]:
best_model = OneVsRestClassifier(MultinomialNB(alpha=0.1))
cross_val_score(best_model, X_train, y_train_genres, cv=10)

array([0.10396975, 0.09640832, 0.11153119, 0.09073724, 0.12287335,
       0.09073724, 0.10586011, 0.12476371, 0.08695652, 0.11742424])

In [21]:
best_model.fit(X_train, y_train_genres)
y_pred_best = best_model.predict_proba(X_test)

# Calcular métricas de desempeño con el mejor modelo
auc_best_model = roc_auc_score(y_test_genres, y_pred_best, average='macro')

print("AUC mejor modelo:", auc_best_model)

AUC mejor modelo: 0.8657962945897676


De acuerdo al apartado anterior, donde se utilizaron diferentes modelos y se utilizo la función GridSearchCV para encontrar los mejores hiperparametros para cada modelo, buscando maximizar el AUC (Área Bajo la Curva), se identifico que los mejores modelos de acuerdo a esa medida de desempeño fueron el modelo de el Clasificador Naive de Bayes y las maquinas de soporte vectorial (SVM). Siendo el Clasificador de bayes el modelo con la mejor medida de desempeño entre todos. Sobre los hiperparametros que se calibraron para este modelo, el mejor modelo es el que tiene un **alpha** de 0.1. 

Luego de calibrado y entrenado, se valida el desempeño del modelo con los datos de validación a través de la métrica AUC . El modelo de Clasificador Naive de Bayes, arroja un **AUC de 0.8658**. Esto significa, que el modelo tiene una alta capacidad para discriminar entre las diferentes clases de géneros de películas y que hay una probabilidad en promedio del **86,58%** de que el modelo pueda distinguir correctamente la pertenencia de una pelicula a uno o varios generos.

## 4. Disponibilización del Modelo
Se disponibiliza el modelo en una API alojada en un servicio en la nube, en una instancia Ubuntu en un microservicio t2 de AWS.

El link para ingresar a la API es el siguiente: [http://18.116.37.110:5000/](http://18.116.37.110:5000/).

La documentación de la API se encuentra alojada en el sigueinte [link](https://github.com/JuanD13Perez/MIAD_gr10_pr2).

Asimismo se cuenta con un [video explicativo del funcionamiento de la API](https://www.youtube.com/watch?v=K440L3_Ke_U).

La API toma el mejor modelo que se encuentra en este Notebook para realizar sus predicciones. Recibe cmomo parámetro un texto que corresponde a la descripción (plot) de la película y devuelve los 3 géneros de película más probables. Su utilizo flask para la estructura de la aplicación.

A continuación añadimos algunas líneas de código que se utilizaron para la disponibiliazación de la API. Los archivos binarios .pkl se encuentran en la carpeta de model_deployment del repositorio en donde se encuentra la documentación de la API. 

In [26]:
# Crear el directorio si no existe
if not os.path.exists('model_deployment'):
    os.makedirs('model_deployment')

# Exportar modelo a archivo binario .pkl
joblib.dump(best_model, 'model_deployment/genremovies.pkl', compress=3)
joblib.dump(vect, 'model_deployment/vectorizer_tfid.pkl', compress = 3)

['model_deployment/genremovies.pkl']

In [ ]:
# Importar modelo y predicción
from model_deployment.model_predictor import predict_genres

# Predicción de probabilidad
predict_genres('A couple begins to experience terrifying supernatural occurrences involving a vintage doll shortly after their home is invaded by satanic cultists. A couple begins to experience terrifying supernatural occurrences involving a vintage doll shortly after their home is invaded by satanic cultists.')

In [ ]:
# Definición aplicación Flask
app = Flask(__name__)

# Definición API Flask
api = Api(
    app, 
    version='1.0', 
    title='API Prediccion generos de peliculas',
    description='API que al ingresar la descripcion de una pelicula (en ingles) predice el genero de pelicula mas probable.')

ns = api.namespace('Predict', 
     description='Clasificador de Generos')

# Definición argumentos o parámetros de la API
parser = api.parser()
parser.add_argument(
    'Descripcion', 
    type=str, 
    required=True, 
    help='Descripcion de pelicula, solo texto (en ingles). Entre comillas simples.', 
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})

In [ ]:
# Definición de la clase para disponibilización
@ns.route('/')
class GenreMovieApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        
        return {
         "result": predict(args['Descripcion'])
        }, 200

In [ ]:
# Ejecución de la aplicación que disponibiliza el modelo de manera local en el puerto 5002
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=500)

In [22]:
# transformación variables predictoras X del conjunto de test
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = best_model.predict_proba(X_test_dtm)

In [23]:
## 4. Disponibilización del Modelo
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv('pred_genres_genres_NV.csv', index_label='ID')
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.038943,0.075070,0.010077,0.010746,0.455676,0.097383,0.004712,0.576995,0.042465,0.155724,...,0.031282,0.043647,0.000100,0.635412,0.008209,0.002045,0.007609,0.165048,0.006099,0.006359
4,0.085844,0.015058,0.002427,0.174447,0.206193,0.247632,0.028262,0.839049,0.007956,0.006308,...,0.005639,0.017758,0.000040,0.084284,0.004618,0.000854,0.015617,0.150881,0.015638,0.009607
5,0.036840,0.002419,0.000883,0.012149,0.236844,0.484512,0.002157,0.792351,0.001855,0.002696,...,0.003198,0.283018,0.000035,0.118200,0.015995,0.000557,0.001982,0.395926,0.002205,0.002461
6,0.059261,0.022946,0.001145,0.008924,0.185501,0.055952,0.003916,0.708321,0.009212,0.010847,...,0.022040,0.020647,0.000014,0.147755,0.034660,0.000228,0.002357,0.212581,0.040467,0.002766
7,0.015121,0.010662,0.003033,0.003935,0.112541,0.067476,0.005704,0.470048,0.011943,0.058729,...,0.004131,0.155601,0.000115,0.159348,0.315366,0.002362,0.001028,0.295962,0.002330,0.004240


## 5. Conclusiones 

Para el conjunto de datos de peliculas con sus respectivas sinopsis en la parte del preprocesamiento se utilizó la técnica de **TfidVectorizer** para vectorizar los textos en números, eliminar las stop words y permitir un análisis por n-gramas. Por último, se divieron los datos en la muestra de entrenamiento y validación. 

Para la calibración, utilizamos la función GridSearchCV cambiando los hiperpárametros de los diferentes modelos que podrían ayudar a dar respuesta al problema de este ejercicio. En ese sentido, el mejor modelo que se obtuvo fue el **Clasificador Naive de Bayes**. Calibrado y entrenado, se válido el desempeño del modelo con los datos de validación a traves del AUC (Area Bajo la Curva). Arrojando un **AUC de 0.8658**, lo que indica que el modelo tiene una alta capacidad para discriminar entre las diferentes clases de géneros de películas y que hay una probabilidad en promedio del **86,58%** de que el modelo pueda distinguir correctamente la pertenencia de una pelicula a uno o varios generos.

Disponibilizamos el modelo en una API Restful. Se disponibilizó a través de un microservicio en la nube en AWS. Para ello creamos dos archivos independientes, con formato tipo "py". El primero llamado "model_predictor.py" que llama el modelo "genremovies.pkl", realiza el preprocesamiento de los datos y realiza las predicciones. El segundo archivo, llamado "api.py" es el que llama la predicción y disponibiliza el modelo. 

Para predecir el o los generos de una pelicula a través de su sinopsis copie en la barra de búsqueda de su navegador la siguiente dirección (http://18.116.37.110:5000/), donde al ingresar y siguiendo las instrucciones puede poner la sinopsis en ingles de las peliculas que desea y esta generara las probabilidades de pertenencia de una película a uno o varios géneros.